In [1]:
import codecs 
import glob 
import multiprocessing 
import os 
import re 
import nltk 
import gensim.models.word2vec as w2v 

D:\Programs\Anaconda\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
book_filenames = glob.glob("data/hp*.txt")
book_filenames

['data\\hp1.txt',
 'data\\hp2.txt',
 'data\\hp3.txt',
 'data\\hp4.txt',
 'data\\hp5.txt',
 'data\\hp6.txt',
 'data\\hp7.txt']

In [3]:
corpus_raw = ""
for book_filename in book_filenames:
    print('Считывается %s' % book_filename)
    with codecs.open(book_filename,'r','utf-8') as file:
        corpus_raw += file.read()
    print('Сборка длиной %d' % len(corpus_raw))
    print()

Считывается data\hp1.txt
Сборка длиной 448811

Считывается data\hp2.txt
Сборка длиной 947731

Считывается data\hp3.txt
Сборка длиной 1569787

Считывается data\hp4.txt
Сборка длиной 2690180

Считывается data\hp5.txt
Сборка длиной 4190905

Считывается data\hp6.txt
Сборка длиной 5182408

Считывается data\hp7.txt
Сборка длиной 6321553



In [4]:
#Разбиваем строку на предложения. Каждое предложение с новой строки
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = tokenizer.tokenize(corpus_raw)
print('Всего %d предложений' % len(raw_sentences))

Всего 85779 предложений


In [5]:
#Конвертируем предложения в массивы слов, убирая знаки пунктуации
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [6]:
#Проверка на пустые строки
sentences = []
for raw_sentence in raw_sentences:
    if(len(raw_sentence) > 0):
        sentences.append(sentence_to_wordlist(raw_sentence))

In [7]:
print('Пример:')
print('Предложение:', raw_sentences[13])
print('Получаем:', sentence_to_wordlist(raw_sentences[13]))

Пример:
Предложение: Mr. Dursley hummed as he picked out his most boring tie for work, and Mrs. Dursley gossiped away happily as she wrestled a screaming Dudley into his high chair.
Получаем: ['Mr', 'Dursley', 'hummed', 'as', 'he', 'picked', 'out', 'his', 'most', 'boring', 'tie', 'for', 'work', 'and', 'Mrs', 'Dursley', 'gossiped', 'away', 'happily', 'as', 'she', 'wrestled', 'a', 'screaming', 'Dudley', 'into', 'his', 'high', 'chair']


In [8]:
token_count = sum([len(sentence) for sentence in sentences])
print('Всего %d токенов' % token_count)

Всего 1120879 токенов


In [9]:
#Размер вектора
num_features = 300

#минимальное пороговое значение
min_word_count = 3

#количество потоков
num_workers = multiprocessing.cpu_count() #возвращает к-во ядер процессора

#размер окна
context_size = 7

#Downsample setting for frequent words
downsampling = 1e-3 #Not see too many times the most frequent words...

#Seed for the randomnessss, to make results reproducible
seed = 1

In [10]:
hp2Vec = w2v.Word2Vec(sg=1,
                           seed=seed,
                           workers=num_workers,
                           size=num_features,
                           min_count=min_word_count,
                           window=context_size,
                           sample=downsampling
)

In [11]:
hp2Vec.build_vocab(sentences)

In [12]:
print('Длина словаря Word2Vec: %d' % hp2Vec.corpus_count)

Длина словаря Word2Vec: 85779


In [13]:
hp2Vec.train(sentences=sentences,total_examples=hp2Vec.corpus_count,epochs=hp2Vec.iter)

4245335

In [14]:
hp2Vec.most_similar("Dumbledore")

[('Lupin', 0.7298069000244141),
 ('Slughorn', 0.7228484153747559),
 ('Firenze', 0.6937481164932251),
 ('Snape', 0.67584228515625),
 ('Fudge', 0.6648882627487183),
 ('headmaster', 0.6583600640296936),
 ('Riddle', 0.6537696123123169),
 ('Scrimgeour', 0.6453474760055542),
 ('Quirrell', 0.6449023485183716),
 ('Aberforth', 0.6334937810897827)]

In [15]:
hp2Vec.most_similar(positive=['Draco', 'Harry'])

[('Lucius', 0.7676020860671997),
 ('Greyback', 0.7372422218322754),
 ('Narcissa', 0.7320736646652222),
 ('Zabini', 0.7319016456604004),
 ('Yaxley', 0.7314563393592834),
 ('encouragingly', 0.728295087814331),
 ('resentfully', 0.7251095175743103),
 ('Travers', 0.7247826457023621),
 ('smirking', 0.7217026948928833),
 ('Amos', 0.7152628302574158)]

In [16]:
hp2Vec.doesnt_match('Harry, Ron, Hermione, Severus'.split())

'Severus'